# Analyzing fire department payrolls in California: 2011-2018

This notebook analyzes government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government.

---

### Load python tools

In [1]:
import pandas as pd
import geopandas as gpd
import cpi
from urllib.request import urlopen 
import pyarrow
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format

### Read controller payroll data

In [2]:
# processed here: 00-california-payroll-processing
src = pd.read_feather('/Users/mhustiles/data/data/controller/output/payroll.feather')

In [3]:
src['year'] = src['year'].astype(str)

In [4]:
payroll = pd.DataFrame(src[(src['year'] != '2009') & (src['year'] != '2010')])

### How many records, years in the dataframe? 

In [5]:
len(payroll)

9756210

In [6]:
payroll.head(1)

,index,year,type,population,employer,department,position,overtime,wages,benefits,employercounty,basewages,adjusted_overtime,adjusted_wages,adjusted_benefits,adjusted_basewages
374773,374773,2017,STATE,0,"ADMINISTRATIVE LAW, OFFICE OF",NOT LISTED,ASSISTANT CHIEF COUNSEL,0,"169,110","46,725",NAN,"169,110",0,"176,379","48,733","176,379"


### Combine employer name, type (ie "Los Angeles" vs. "Los Angeles County"

In [7]:
payroll['employerfull'] = (payroll['employer'] + '_' + payroll['type']).str.replace(' ','_')

### Read in our processed table of fire titles, convert to a list

In [8]:
positions = pd.read_csv('input/positions.csv')

In [9]:
positions_list = positions[positions['category'] == 'firefighter']['position'].tolist()

### Tag each record it the employee's title matches our 'firefighter' list

In [10]:
# payroll['fire'] = \
#     (payroll['position'].isin(positions_list) & payroll['department'].str.contains('FIRE')) | \
#     (payroll['position'].isin(positions_list) & payroll['employer'].str.contains('FIRE'))

In [11]:
payroll_fire = pd.DataFrame(payroll[(payroll['position'].isin(positions_list) &\
                                     payroll['department'].str.contains('FIRE')) |\
                                    (payroll['position'].isin(positions_list) &\
                                     payroll['employer'].str.contains('FIRE')) |\
                                   (payroll['position'].isin(positions_list) &\
                                     payroll['position'].str.contains('FIRE'))])

In [12]:
len(payroll_fire)

347574

### Read just the 'firefighter' payroll records that included some amount of overtime into a dataframe

In [13]:
# payroll_fire = pd.DataFrame(payroll[(payroll['fire'] == True)])
# non_payroll_fire = pd.DataFrame(payroll[(payroll['fire'] == False)])

### How many fire agenies

In [14]:
entities = payroll_fire.groupby(['employer']).agg('size')
years = payroll_fire.groupby(['year']).agg('size')

In [15]:
years.head()

year
2011    37726
2012    37671
2013    37562
2014    37417
2015    38441
dtype: int64

In [16]:
len(entities)

586

### Export fire payroll dataframe to csv 

In [17]:
payroll_fire.to_csv('/Users/mhustiles/data/data/controller/output/payroll_fire.csv')

---

### Statewide aggregates

In [18]:
state = payroll_fire.groupby(['year']).agg({'adjusted_overtime':'sum', \
                                            'adjusted_overtime':'sum', \
                                            'adjusted_wages':'sum', \
                                            'adjusted_basewages':'sum', \
                                            'adjusted_benefits':'sum'}).reset_index()

In [19]:
state.head(8)

,year,adjusted_overtime,adjusted_wages,adjusted_basewages,adjusted_benefits
0,2011,"838,877,369","4,182,727,880","3,343,850,511","1,294,307,252"
1,2012,"951,936,641","4,255,059,134","3,303,122,494","1,346,184,409"
2,2013,"1,021,348,278","4,264,986,368","3,243,638,090","1,403,209,497"
3,2014,"1,094,546,068","4,268,715,795","3,174,169,727","1,464,576,166"
4,2015,"1,221,411,273","4,619,236,939","3,397,825,665","1,577,961,644"
5,2016,"1,269,955,539","4,770,777,111","3,500,821,572","1,648,770,858"
6,2017,"1,354,261,182","4,905,734,068","3,551,472,886","1,542,622,657"
7,2018,"1,388,759,908","4,989,322,946","3,600,563,038","1,610,651,254"


### How much has overtime increased statewide? 

In [20]:
'{:,.0f}%'.format((((state.iloc[7,1] - state.iloc[0,1]) / state.iloc[0,1])*100).round(2))

'66%'

In [21]:
'${:,.0f}'.format((state.iloc[7,1] - state.iloc[0,1]))

'$549,882,539'

In [22]:
state_bar = alt.Chart(state).mark_bar().encode(
    x=alt.X('year:N', axis=alt.Axis(format='', tickCount=5)),
    y=alt.Y("sum(adjusted_overtime):Q", title=" ", axis=alt.Axis(format='$,N', tickCount=6))
)

In [23]:
state_bar_text = state_bar.mark_text(
    align='center',
    baseline='top',
    dx=0,
    dy=-20 # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('sum(adjusted_overtime):Q', format="$,.3s"))

In [24]:
(state_bar + state_bar_text)\
.properties(height=400, width=600, title='Statewide firefighter overtime spending, 2011-19')

alt.LayerChart(...)

---

### Group by employer/type and sum overtime expenses by year

In [25]:
payroll_fire_mean = payroll_fire.groupby(['year', 'type']).agg({'adjusted_overtime':'mean', 'adjusted_basewages':'mean'}).reset_index()

In [26]:
payroll_fire_mean.head(100)

,year,type,adjusted_overtime,adjusted_basewages
0,2011,CITY,"23,962","105,657"
1,2011,COUNTY,"33,913","89,702"
2,2011,SPECIAL DISTRICT,"14,664","65,147"
3,2011,STATE,"12,787","59,215"
4,2012,CITY,"27,840","104,543"
5,2012,COUNTY,"33,225","82,974"
6,2012,SPECIAL DISTRICT,"15,588","63,790"
7,2012,STATE,"18,575","64,370"
8,2013,CITY,"30,863","103,929"
9,2013,COUNTY,"36,545","82,445"


### Pivot on employer type and widen the dataframe across the years

In [27]:
payroll_fire_mean_pivot = pd.pivot_table(payroll_fire_mean, \
                            values='adjusted_overtime', index=['type'], columns=['year']).reset_index().fillna(0)

In [28]:
payroll_fire_mean_pivot.columns = payroll_fire_mean_pivot.columns.map(str)

In [29]:
payroll_fire_mean_pivot.head()

year,type,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,CITY,"23,962","27,840","30,863","33,081","36,494","36,335","37,891","38,254","35,373"
1,COUNTY,"33,913","33,225","36,545","38,581","42,511","46,739","48,338","47,438","45,746"
2,SPECIAL DISTRICT,"14,664","15,588","15,185","17,292","18,659","18,994","22,065","22,663","22,005"
3,STATE,"12,787","18,575","19,100","21,572","21,863","20,058","23,871","27,120","19,357"


### Trim the data set to early, middle and current years

In [30]:
fire_pivot = payroll_fire_mean_pivot[['type', '2011','2014','2019']].reset_index()

### Select only those places that had some over time

In [31]:
fire_pivot = fire_pivot[(fire_pivot['2011'] > 1) &\
                        (fire_pivot['2014'] > 1) &\
                        (fire_pivot['2019'] > 1)]

In [32]:
fire_pivot = fire_pivot[['type','2011','2014','2019']]

### What was the percentage increase in adjusted overtime, by place?

In [33]:
fire_pivot['increase_adjusted_pct'] = (((fire_pivot['2019'] - \
                                    fire_pivot['2011'])/fire_pivot['2011'])*100).astype(int)

In [34]:
fire_pivot.sort_values(by='increase_adjusted_pct', ascending=False).head(10)

year,type,2011,2014,2019,increase_adjusted_pct
3,STATE,"12,787","21,572","19,357",51
2,SPECIAL DISTRICT,"14,664","17,292","22,005",50
0,CITY,"23,962","33,081","35,373",47
1,COUNTY,"33,913","38,581","45,746",34


In [35]:
fire_pivot.to_csv('output/fire_pivot.csv')

### How much did overtime, in adjusted dollars, go up for each agency type?

In [36]:
employers_grouped = payroll_fire.groupby(['type','year']).agg({'overtime': 'mean'}).reset_index()

In [37]:
employers_grouped.dtypes

type         object
year         object
overtime    float64
dtype: object

In [38]:
chart = alt.Chart(employers_grouped).mark_bar().encode(
    x=alt.X('year:N', title='', axis=alt.Axis(format='', tickCount=6)),
    y=alt.Y('overtime:Q', title='', axis=alt.Axis(format='$,N', tickCount=6)),
    facet=alt.Facet('type:N', title='Average firefighter overtime, by CA fire agency type and year')
).properties(width=250,height=250)

In [39]:
chart.configure_header(
    titleColor='Black',
    titleFontSize=18,
    labelColor='Black',
    labelFontSize=11
)

alt.Chart(...)

### How much has each agency spent — in CPI-inflated dollars — on overtime in the past decade? 

In [40]:
agency_overtime = payroll_fire.groupby(['employerfull', 'type', 'employercounty','year'])\
                    .agg({'adjusted_overtime':'sum', \
                          'adjusted_wages':'sum',\
                          'adjusted_basewages':'sum',\
                          'overtime':'sum', \
                          'wages':'sum'}).reset_index()

In [41]:
agency_overtime.head()

,employerfull,type,employercounty,year,adjusted_overtime,adjusted_wages,adjusted_basewages,overtime,wages
0,ALAMEDA_CITY,CITY,ALAMEDA,2011,"803,203","14,023,072","13,219,868","706,696","12,338,155"
1,ALAMEDA_CITY,CITY,ALAMEDA,2012,"557,327","13,023,898","12,466,571","500,510","11,696,174"
2,ALAMEDA_CITY,CITY,ALAMEDA,2013,"638,283","13,825,347","13,187,065","581,609","12,597,783"
3,ALAMEDA_CITY,CITY,ALAMEDA,2014,"1,259,160","14,540,816","13,281,656","1,165,970","13,464,660"
4,ALAMEDA_CITY,CITY,ALAMEDA,2015,"2,342,254","15,850,867","13,508,612","2,171,480","14,695,177"


### What's the share of each agency's wages that is overtime?

In [42]:
agency_overtime['ot_share_wages'] = ((agency_overtime['overtime'] /\
                                      agency_overtime['wages'])*100).round(2)

In [43]:
agency_overtime = pd.DataFrame(agency_overtime\
#                                [agency_overtime['adjusted_overtime'] > 0]\
                              )

In [44]:
agency_overtime.to_csv('/Users/mhustiles/data/data/controller/output/agency_overtime.csv')

### What about agencies in our area? 

In [45]:
area_counties = ['LOS ANGELES', 'ORANGE', 'KERN', 'VENTURA', 'RIVERSIDE', 'SAN BERNARDINO']

In [46]:
la_agency_overtime = agency_overtime[(agency_overtime['employercounty'] == 'LOS ANGELES') &\
                                         (agency_overtime['type'] != 'SPECIAL DISTRICT')]

In [47]:
la_area_agency_overtime = agency_overtime[(agency_overtime['employercounty'].isin(area_counties)) &\
                                         (agency_overtime['type'] != 'SPECIAL DISTRICT')]

In [48]:
la_agency_overtime.head(5)

,employerfull,type,employercounty,year,adjusted_overtime,adjusted_wages,adjusted_basewages,overtime,wages,ot_share_wages
26,ALHAMBRA_CITY,CITY,LOS ANGELES,2011,"2,422,368","8,747,377","6,325,009","2,131,313","7,696,352",28
27,ALHAMBRA_CITY,CITY,LOS ANGELES,2012,"2,620,875","8,626,075","6,005,200","2,353,689","7,746,688",30
28,ALHAMBRA_CITY,CITY,LOS ANGELES,2013,"2,478,488","8,716,864","6,238,376","2,258,421","7,942,886",28
29,ALHAMBRA_CITY,CITY,LOS ANGELES,2014,"2,548,421","8,531,589","5,983,168","2,359,814","7,900,172",30
30,ALHAMBRA_CITY,CITY,LOS ANGELES,2015,"2,214,889","8,262,582","6,047,693","2,053,401","7,660,156",27


In [49]:
agency_overtime[agency_overtime.employerfull.str.contains('')].\
    sort_values(by='overtime', ascending=False).head(10)

,employerfull,type,employercounty,year,adjusted_overtime,adjusted_wages,adjusted_basewages,overtime,wages,ot_share_wages
2223,LOS_ANGELES_CITY,CITY,LOS ANGELES,2019,"207,856,755","628,542,751","420,685,996","207,856,755","628,542,751",33
2232,LOS_ANGELES_COUNTY,COUNTY,LOS ANGELES,2019,"206,244,625","552,503,321","346,258,696","206,244,625","552,503,321",37
2231,LOS_ANGELES_COUNTY,COUNTY,LOS ANGELES,2018,"203,838,869","547,049,838","343,210,969","200,211,091","537,313,837",37
2230,LOS_ANGELES_COUNTY,COUNTY,LOS ANGELES,2017,"206,097,766","544,509,701","338,411,935","197,603,369","522,067,528",38
2221,LOS_ANGELES_CITY,CITY,LOS ANGELES,2017,"204,805,434","612,039,575","407,234,140","196,364,301","586,814,132",33
2222,LOS_ANGELES_CITY,CITY,LOS ANGELES,2018,"198,411,726","606,685,551","408,273,826","194,880,536","595,888,197",33
2229,LOS_ANGELES_COUNTY,COUNTY,LOS ANGELES,2016,"200,750,586","529,889,247","329,138,660","188,461,673","497,452,166",38
2220,LOS_ANGELES_CITY,CITY,LOS ANGELES,2016,"198,223,610","583,796,306","385,572,696","186,089,385","548,059,314",34
2219,LOS_ANGELES_CITY,CITY,LOS ANGELES,2015,"199,022,322","582,418,739","383,396,417","184,511,567","539,954,479",34
506,CALFIRE_STATE,STATE,NAN,2018,"185,538,996","686,704,325","501,165,329","182,236,906","674,482,853",27


In [50]:
agency_overtime[agency_overtime.type == 'CITY'].sort_values(by='adjusted_overtime', ascending=False).head(10)

,employerfull,type,employercounty,year,adjusted_overtime,adjusted_wages,adjusted_basewages,overtime,wages,ot_share_wages
2223,LOS_ANGELES_CITY,CITY,LOS ANGELES,2019,"207,856,755","628,542,751","420,685,996","207,856,755","628,542,751",33
2221,LOS_ANGELES_CITY,CITY,LOS ANGELES,2017,"204,805,434","612,039,575","407,234,140","196,364,301","586,814,132",33
2219,LOS_ANGELES_CITY,CITY,LOS ANGELES,2015,"199,022,322","582,418,739","383,396,417","184,511,567","539,954,479",34
2222,LOS_ANGELES_CITY,CITY,LOS ANGELES,2018,"198,411,726","606,685,551","408,273,826","194,880,536","595,888,197",33
2220,LOS_ANGELES_CITY,CITY,LOS ANGELES,2016,"198,223,610","583,796,306","385,572,696","186,089,385","548,059,314",34
2218,LOS_ANGELES_CITY,CITY,LOS ANGELES,2014,"174,201,837","514,491,846","340,290,009","161,309,278","476,414,656",34
2217,LOS_ANGELES_CITY,CITY,LOS ANGELES,2013,"154,272,930","534,463,557","380,190,626","140,574,907","487,008,088",29
2216,LOS_ANGELES_CITY,CITY,LOS ANGELES,2012,"125,749,783","511,540,078","385,790,295","112,930,198","459,391,030",25
2215,LOS_ANGELES_CITY,CITY,LOS ANGELES,2011,"109,199,314","514,016,858","404,817,544","96,078,670","452,256,101",21
3655,SAN_FRANCISCO_CITY,CITY,SAN FRANCISCO,2013,"48,060,755","256,643,251","208,582,496","43,793,400","233,855,681",19


In [51]:
agency_overtime.to_csv('output/agency_overtime_share_ot.csv')

In [52]:
novato = agency_overtime[agency_overtime['employerfull'].str.contains('NOVATO')]

In [53]:
novato.head(10)

,employerfull,type,employercounty,year,adjusted_overtime,adjusted_wages,adjusted_basewages,overtime,wages,ot_share_wages
2837,NOVATO_FIRE_PROTECTION_DISTRICT_SPECIAL_DISTRICT,SPECIAL DISTRICT,MARIN,2011,"1,213,118","7,245,158","6,032,040","1,067,358","6,374,629",17
2838,NOVATO_FIRE_PROTECTION_DISTRICT_SPECIAL_DISTRICT,SPECIAL DISTRICT,MARIN,2012,"1,572,424","6,504,211","4,931,787","1,412,123","5,841,138",24
2839,NOVATO_FIRE_PROTECTION_DISTRICT_SPECIAL_DISTRICT,SPECIAL DISTRICT,MARIN,2013,"2,268,955","9,772,383","7,503,428","2,067,493","8,904,685",23
2840,NOVATO_FIRE_PROTECTION_DISTRICT_SPECIAL_DISTRICT,SPECIAL DISTRICT,MARIN,2014,"2,435,562","10,452,658","8,017,096","2,255,308","9,679,064",23
2841,NOVATO_FIRE_PROTECTION_DISTRICT_SPECIAL_DISTRICT,SPECIAL DISTRICT,MARIN,2015,"3,071,860","11,025,250","7,953,390","2,847,890","10,221,397",28
2842,NOVATO_FIRE_PROTECTION_DISTRICT_SPECIAL_DISTRICT,SPECIAL DISTRICT,MARIN,2016,"3,617,766","11,539,930","7,922,164","3,396,305","10,833,515",31
2843,NOVATO_FIRE_PROTECTION_DISTRICT_SPECIAL_DISTRICT,SPECIAL DISTRICT,MARIN,2017,"4,014,726","11,914,247","7,899,521","3,849,258","11,423,197",34
2844,NOVATO_FIRE_PROTECTION_DISTRICT_SPECIAL_DISTRICT,SPECIAL DISTRICT,MARIN,2018,"3,937,495","11,634,550","7,697,056","3,867,418","11,427,487",34
2845,NOVATO_FIRE_PROTECTION_DISTRICT_SPECIAL_DISTRICT,SPECIAL DISTRICT,MARIN,2019,"3,306,306","11,378,702","8,072,396","3,306,306","11,378,702",29


In [54]:
novato.iloc[7,4] 

3937494.707937253

In [55]:
'{:,.0f}%'.format((((novato.iloc[7,4] - novato.iloc[0,4]) / novato.iloc[0,4])*100).round(2))

'225%'

---

### Slice out specific large employer to CSVs to share for interviews

adjust this so it only gets the biggest departments, not just the biggest cities, counties

In [56]:
agencies_totals = agency_overtime.groupby('employerfull').agg({'adjusted_overtime':'sum'})\
    .sort_values(by='adjusted_overtime', ascending=False).reset_index().head(30)

In [57]:
agencies_totals

,employerfull,adjusted_overtime
0,LOS_ANGELES_COUNTY,"1,614,796,620"
1,LOS_ANGELES_CITY,"1,571,743,711"
2,CALFIRE_STATE,"1,194,523,149"
3,OC_FIRE_AUTHORITY_SPECIAL_DISTRICT,"403,808,457"
4,SAN_FRANCISCO_CITY,"360,208,960"
5,SAN_DIEGO_CITY,"323,278,734"
6,KERN_COUNTY,"193,652,296"
7,VENTURA_COUNTY,"179,788,937"
8,SACRAMENTO_METRO_FIRE_SPECIAL_DISTRICT,"177,971,688"
9,OAKLAND_CITY,"165,893,983"


In [58]:
lg_agencies_list = list(agencies_totals.employerfull)

In [59]:
lg_agencies_list = ['LOS_ANGELES_COUNTY',
 'LOS_ANGELES_CITY',
 'CALFIRE_STATE',
 'OC_FIRE_AUTHORITY_SPECIAL_DISTRICT',
 'SAN_FRANCISCO_CITY',
 'SAN_DIEGO_CITY',
 'KERN_COUNTY',
 'VENTURA_COUNTY',
 'SACRAMENTO_METRO_FIRE_SPECIAL_DISTRICT',
 'LONG_BEACH_CITY',
 'OAKLAND_CITY',
 'SACRAMENTO_CITY',
 'CONTRA_COSTA_COUNTY',
 'SAN_JOSE_CITY',
 'ANAHEIM_CITY',
 'SANTA_BARBARA_COUNTY',
 'PASADENA_CITY',
 'GLENDALE_CITY',
 'SAN_RAMON_VALLEY_FIRE_PROTECTION_SPECIAL_DISTRICT',
 'RIVERSIDE_CITY',
 'ALAMEDA_COUNTY',
 'FRESNO_CITY',
 'ONTARIO_CITY',
 'SANTA_MONICA_CITY',
 'HUNTINGTON_BEACH_CITY',
 'CORONA_CITY',
 'SANTA_CLARA_COUNTY',
 'FREMONT_CITY',
 'CHULA_VISTA_CITY',
 'NEWPORT_BEACH_CITY',
'EL_SEGUNDO_CITY']

In [60]:
large_places = payroll_fire[(payroll_fire['population'] > 100000)]\
.groupby('employerfull').agg('size').reset_index(name='count')

In [61]:
large_places.head(5)

,employerfull,count
0,ALAMEDA_COUNTY,1479
1,ANAHEIM_CITY,1948
2,BAKERSFIELD_CITY,1647
3,BERKELEY_CITY,1187
4,BURBANK_CITY,1104


In [62]:
locations = []
for p in lg_agencies_list:
    locations.append(dict(location = p))

In [63]:
df = pd.DataFrame()

for l in locations:
    d = l['location']
    place = payroll_fire[payroll_fire['employerfull'] == d]
    place.to_csv(f'output/places/' + d.lower() + '_employee_list' + '.csv' , sep=',', index=False)
    place_slim = pd.DataFrame(place[['employer', 'type', 'overtime', 'wages', 'adjusted_overtime', 'year']])
    place_slim_grouped = place_slim.groupby(['employer', 'type', 'year'])\
                .agg({'overtime': 'sum', 'adjusted_overtime': 'sum', 'wages': 'sum'}).reset_index()
    place_slim_grouped.to_csv(f'output/places/' + d.lower() + '_summary' + '.csv' , sep=',', index=False)
    place_slim_grouped_pivot = pd.pivot_table(place_slim_grouped, \
                values='adjusted_overtime', index=['employer', 'type'], columns=['year']).reset_index().fillna(0)
    place_slim_grouped_pivot_slim = pd.DataFrame(place_slim_grouped_pivot[['employer','type','2011','2018']])
    place_slim_grouped_pivot_slim['adjusted_ot_pct_change'] = \
                ((place_slim_grouped_pivot_slim['2018']-place_slim_grouped_pivot_slim['2011'])/\
                 place_slim_grouped_pivot_slim['2011'])*100
    place_slim_grouped_pivot_slim.to_csv(f'output/places/' + d.lower() + '_pivot' + '.csv' , sep=',', index=False)

### Create a dataframe combining and pivoting listing all the large places

In [64]:
large_places_df = payroll_fire[(payroll_fire['population'] > 10000) \
                               & (payroll_fire['employer'] != 'FRESNO')].groupby(['employer', 'type', 'year'])\
                .agg({'adjusted_overtime': 'sum'}).reset_index()

In [65]:
large_places_df_pivot = pd.DataFrame(pd.pivot_table(large_places_df, \
                values='adjusted_overtime', index=['employer', 'type'], columns=['year']).reset_index().fillna(0))

In [66]:
large_places_df_pivot['adjusted_ot_pct_change'] = \
                ((large_places_df_pivot['2019']-large_places_df_pivot['2011'])/\
                 large_places_df_pivot['2011'])*100

In [67]:
places = large_places_df_pivot[(large_places_df_pivot['2011'] > 10000) & (large_places_df_pivot['2019'] > 10000)]

In [68]:
large_places_df_pivot.head()

year,employer,type,2011,2012,2013,2014,2015,2016,2017,2018,2019,adjusted_ot_pct_change
0,ALAMEDA,CITY,"803,203","557,327","638,283","1,259,160","2,342,254","1,370,063","1,443,485","2,032,825","2,676,033",233
1,ALAMEDA,COUNTY,0,0,0,0,"14,208,990","16,152,871","3,503,457","17,187,372","16,327,468",inf
2,ALBANY,CITY,"582,145","784,939","649,931","615,708","660,770",0,"809,076","941,092","913,016",57
3,ALHAMBRA,CITY,"2,422,368","2,620,875","2,478,488","2,548,421","2,214,889","3,013,562","3,504,914","3,780,845","2,818,008",16
4,AMADOR,COUNTY,0,0,0,0,0,0,"510,179","470,262","401,231",inf


In [69]:
large_places_df_pivot['adjusted_ot_pct_change'] = \
                ((large_places_df_pivot['2019']-large_places_df_pivot['2011'])/\
                 large_places_df_pivot['2014'])*100

In [70]:
places = large_places_df_pivot[(large_places_df_pivot['2014'] > 0) & (large_places_df_pivot['2019'] > 0)]

In [71]:
places.sort_values(by='2019', ascending=False).head()

year,employer,type,2011,2012,2013,2014,2015,2016,2017,2018,2019,adjusted_ot_pct_change
92,LOS ANGELES,CITY,"109,199,314","125,749,783","154,272,930","174,201,837","199,022,322","198,223,610","204,805,434","198,411,726","207,856,755",57
93,LOS ANGELES,COUNTY,"143,138,158","147,633,267","162,505,150","165,835,905","178,752,293","200,750,586","206,097,766","203,838,869","206,244,625",38
165,SAN DIEGO,CITY,"22,423,000","34,272,611","34,778,859","34,848,758","33,382,613","35,550,931","40,670,330","44,342,226","43,009,407",59
167,SAN FRANCISCO,CITY,"33,152,175","44,611,341","48,060,755","40,919,023","42,820,606","44,492,970","36,227,255","34,856,055","35,068,780",5
125,OAKLAND,CITY,"187,966","16,571,623","18,161,242","22,341,243","23,179,979","23,263,548","20,376,972","19,384,231","22,427,180",100


In [72]:
places.to_csv('output/places/large_places.csv')

---

### Aggregating agencies by county

In [73]:
counties = agency_overtime[agency_overtime['employercounty'] != 'NAN'].\
                           groupby(['employercounty', 'year']).agg({'adjusted_overtime':'sum'}).reset_index()

In [74]:
counties_pivot = pd.pivot_table(counties, \
                            values='adjusted_overtime',\
                            index=['employercounty'], \
                            columns=['year']).reset_index().fillna(0)

In [75]:
counties_pivot.columns = counties_pivot.columns.map(str)

In [76]:
counties_pivot.to_csv('output/counties_pivot.csv')

In [77]:
counties_pivot_trim = pd.DataFrame(counties_pivot[(counties_pivot['2011'] > 0) & (counties_pivot['2019'] > 0)])

In [78]:
counties_pivot_trim['pct_increase'] = ((((counties_pivot_trim['2019'] - counties_pivot_trim['2011']) \
                               / counties_pivot_trim['2011'])*100).round(2)).astype(int)

In [79]:
counties_pivot_trim.sort_values(by='pct_increase', ascending=False).head(10)

year,employercounty,2011,2012,2013,2014,2015,2016,2017,2018,2019,pct_increase
46,SISKIYOU,589,"2,499","10,995","98,201","123,143","133,053","22,190","63,410","42,048",7042
17,LASSEN,"11,060","82,916","61,651","60,923","81,526","79,224","72,508","134,199","109,819",892
2,AMADOR,"51,839","6,889","79,018","87,354","77,556","67,548","557,894","577,878","496,375",857
0,ALAMEDA,"13,115,421","32,949,541","31,405,543","35,227,669","57,706,039","58,608,590","44,373,626","58,631,691","60,332,524",360
34,SAN BENITO,"329,078","248,813","533,575","362,562","313,089","649,259","1,111,315","1,636,208","1,454,117",341
57,YUBA,"108,668","163,692","218,380","273,936","355,351","356,981","620,753","836,230","441,818",306
9,FRESNO,"3,411,184","4,253,048","6,162,108","8,135,120","11,441,704","12,347,859","13,087,641","14,290,071","11,052,194",224
15,KINGS,"688,088","762,687","1,035,041","1,688,639","1,837,310","1,998,167","2,439,275","2,581,892","2,169,583",215
39,SAN LUIS OBISPO,"1,053,795","1,175,645","3,463,727","3,497,022","3,596,145","4,267,473","4,448,278","4,453,278","3,315,199",214
47,SOLANO,"3,566,697","6,153,356","5,459,485","7,366,730","8,792,127","8,503,254","11,508,399","12,037,461","10,580,617",196


In [80]:
counties_pivot_trim['year2019'] = (counties_pivot_trim['2019']).round(0).astype(int)

In [81]:
counties_pivot_trim.dtypes

year
employercounty     object
2011              float64
2012              float64
2013              float64
2014              float64
2015              float64
2016              float64
2017              float64
2018              float64
2019              float64
pct_increase        int64
year2019            int64
dtype: object

In [82]:
breaks_increase = jenkspy.jenks_breaks(list(counties_pivot_trim.pct_increase), nb_class=7)

In [83]:
breaks_2018 = jenkspy.jenks_breaks(list(counties_pivot_trim.year2018), nb_class=7)

AttributeError: 'DataFrame' object has no attribute 'year2018'

In [ ]:
def group_breaks_increase(breaks_increase_value):
    for i, b in enumerate(breaks_increase):
        if breaks_increase_value <= breaks_increase[i+1]:
            return i

In [ ]:
def get_group(value):
    for i, b in enumerate(breaks_2018):
        if value <= breaks_2018[i+1]:
            return i

In [ ]:
counties_pivot_trim['increase_group'] = counties_pivot_trim.pct_increase.apply(group_breaks_increase)

In [ ]:
counties_pivot_trim['2018_group'] = counties_pivot_trim.year2018.apply(get_group)

In [ ]:
counties_pivot_trim.head(10)

### Add counties geodataframe

In [ ]:
counties = gpd.read_file('http://s3-us-west-2.amazonaws.com/boundaries.\
latimes.com/archive/1.0/boundary-set/counties-2012.geojson')

In [ ]:
counties.plot()

In [ ]:
counties = counties.rename(columns={'name':'employercounty'})
counties['employercounty'] = counties['employercounty'].str.upper()

In [ ]:
counties_overtime = counties.merge(counties_pivot_trim, on='employercounty', how='left')

In [ ]:
counties_overtime.head()

In [ ]:
lat.palette['schemes'] = {
    'div_fire': ['#fdd0a2','#dadaeb','#bcbddc','#9e9ac8','#807dba','#6a51a3','#4a1486'],
    'category-6': ['#3580b1','#ec8431','#ab7fb4','#c89d29','#adc839','#829eb1'],
    'category-6': ['#3580b1','#ec8431','#ab7fb4','#c89d29','#adc839','#829eb1'],
    'fire-7': ['#fbf2c7','#f9e39c','#f8d36e','#f4bb6a','#e68a4f','#d15a40','#ab4232'],
    'fireandice-6': ['#e68a4f','#f4bb6a','#f9e39c','#dadfe2','#a6b7c6','#849eae'],
    'ice-7': ['#edefee','#dadfe2','#c4ccd2','#a6b7c6','#849eae','#607785','#47525d']}

In [ ]:
counties_overtime.to_file('output/counties_overtime.geojson', driver='GeoJSON')

In [ ]:
geojson = json.loads(counties_overtime.to_json())

In [ ]:
features = alt.Data(values=geojson['features'])

In [ ]:
geoshape = alt.Chart(features).mark_geoshape(fill='#e6e6e6', stroke='white')

In [ ]:
base = alt.Chart(features).mark_geoshape( fill='#e6e6e6', stroke='white', strokeWidth=0.1)\
        .encode().properties(
    width=400,
    height=500
)

In [ ]:
increase = geoshape.encode( color=alt.Color( "properties.increase_group:N", 
      scale=alt.Scale(
      domain=[0,1, 2, 3, 4, 5, 6],
      range=lat.palette['schemes']['div_fire']
        ),legend=None),)

In [ ]:
year2018 = geoshape.encode( color=alt.Color( "properties.2018_group:N", 
      scale=alt.Scale(
      domain=[0,1, 2, 3, 4, 5, 6],
      range=lat.palette['schemes']['fire-7']
        ),legend=None),)

In [ ]:
change = (base + increase).properties(title="Increase in overtime spending")

In [ ]:
total = (base + year2018).properties(title="Overtime spending, by county")

In [ ]:
total | change

---

In [ ]:
labars = pd.DataFrame(la_agency_overtime[['employerfull', 'year', 'ot_share_wages']])
labars.head()

In [ ]:
alt.Chart(labars).mark_area().encode(
    x=alt.X('year:N', title='', axis=alt.Axis(format='', tickCount=2, values=(2011,2019))),
    y=alt.Y("ot_share_wages:Q", title="", axis=alt.Axis(format='', tickCount=10)),
    facet=alt.Facet('employerfull:N', columns=7)
).properties(height=100, width=100, title='')

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx

---

### How many firefighters outside L.A. received six-figure salaries? 

In [ ]:
state_high = payroll_fire[(payroll_fire['overtime'] > 100000) &\
                             (~payroll_fire['employer'].str.contains('LOS ANGELES'))].reset_index()

In [ ]:
state_highest = payroll_fire[(payroll_fire['overtime'] > 200000) &\
                             (~payroll_fire['employer'].str.contains('LOS ANGELES'))].reset_index()

In [ ]:
print('$200k:',len(state_highest),'| $100k:',len(state_high))

In [ ]:
state_high.sort_values(by='adjusted_overtime', ascending=False).head()

In [ ]:
state_highest_years = state_highest.groupby(['year', 'type']).agg('size').reset_index(name='count')
state_highest_years.sort_values(by='count', ascending=False).head()

In [ ]:
state_high_years = state_high.groupby(['year', 'type']).agg('size').reset_index(name='count')
state_high_years.sort_values(by='count', ascending=False).head()

In [ ]:
alt.Chart(state_high_years).mark_bar().encode(
    x=alt.X("year:N", title=' ', axis=alt.Axis(values=(2011, 2019))),
    y=alt.Y("count:Q", title=' ', axis=alt.Axis(tickCount=6, format='2f')),
    facet='type:O'
).properties(width=200, height=200, title='Calif. $100,000 overtime firefighters, by agency type')

---

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx